In [1]:
!pip install pdfplumber
!pip install python-certifi-win32
!pip install certifi
!pip install --upgrade certifi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.6 MB/s eta 0:00:00


In [2]:
import requests
from io import BytesIO
import pdfplumber
import ssl
from bs4 import BeautifulSoup
import json
import os
import re
from typing import List, Dict


In [3]:
def save_uri_in_json(uri):
  file_name = 'uris.json'
  if os.path.exists(file_name):
      with open(file_name, 'r', encoding='utf-8') as json_file:
          data = json.load(json_file)
  else:
      data = {"uris": {}}


  data["uris"][uri] = "pending"

  with open(file_name, 'w', encoding='utf-8') as json_file:
      json.dump(data, json_file, indent=4, ensure_ascii=False)

In [4]:
def save_metadata_in_json(uri: str, data_dict: Dict[str, dict]):
    file_name = 'dataset.json'

    try:
        try:
            with open(file_name, 'r', encoding='utf-8') as json_file:
                existing_data = json.load(json_file)
        except (FileNotFoundError, json.JSONDecodeError):
            existing_data = {}


        existing_data[uri] = data_dict

        with open(file_name, 'w',  encoding='utf-8') as json_file:
            json.dump(existing_data, json_file, indent=4, ensure_ascii=False)

    except Exception as e:
        print(f"ERROR: {e}")

In [5]:
def save_uris_pdfs_and_metadata_from_repositorio_unal():
  base_url = "https://repositorio.unal.edu.co/handle/unal/5/recent-submissions?offset="
  offset = 0
  url_list = []
  while offset <= 1900:
      url = base_url + str(offset)
      print('OFFSET: '+ str(offset))
      offset += 20
      try:
        response = requests.get(url, verify=False)
        soup = BeautifulSoup(response.content, 'html.parser')

        artifact_items = soup.find_all(class_="ds-artifact-item")

        for item in artifact_items:
            item_wrapper = item.find('div', class_="item-wrapper row")
            if item_wrapper:
                col_div = item_wrapper.find('div', class_="col-sm-3 hidden-xs")

                if col_div:
                    thumbnail_div = col_div.find('div', class_="thumbnail artifact-preview")
                    if thumbnail_div:
                        a_tag = thumbnail_div.find('a', class_="image-link")

                        if a_tag and 'href' in a_tag.attrs:
                            item_page_url = a_tag['href']
                            item_page_url = item_page_url.strip()
                            item_page_url = 'https://repositorio.unal.edu.co' + item_page_url + '?show=full'
                            item_page_response = requests.get(item_page_url, verify=False)
                            item_page_soup = BeautifulSoup(item_page_response.content, 'html.parser')

                            advisor_label = item_page_soup.find('td', class_='label-cell', string='dc.contributor.advisor')
                            if advisor_label:
                                advisor_row = advisor_label.find_parent('tr')
                                advisor = str(advisor_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            author_label = item_page_soup.find('td', class_='label-cell', string='dc.contributor.author')
                            if author_label:
                                author_row = author_label.find_parent('tr')
                                author = str(author_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            date_label = item_page_soup.find('td', class_='label-cell', string='dc.date.issued')
                            if date_label:
                                date_row = date_label.find_parent('tr')
                                date = str(date_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')

                            description_label = item_page_soup.find('td', class_='label-cell', string='dc.description.abstract')
                            if description_label:
                                description_row = description_label.find_parent('tr')
                                description = str(description_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')

                            title_label = item_page_soup.find('td', class_='label-cell', string='dc.title')
                            if title_label:
                                title_row = title_label.find_parent('tr')
                                title = str(title_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            program_label = item_page_soup.find('td', class_='label-cell', string='dc.publisher.program')
                            if program_label:
                                program_row = program_label.find_parent('tr')
                                program = str(program_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            faculty_label = item_page_soup.find('td', class_='label-cell', string='dc.publisher.faculty')
                            if faculty_label:
                                faculty_row = faculty_label.find_parent('tr')
                                faculty = str(faculty_row.find_all('td', class_='word-break')[0]).replace('<td class="word-break">', '').replace('</td>','')


                            data_dict = {
                                "advisor": advisor,
                                "author": author,
                                "date": date,
                                "description": description,
                                "title": title,
                                "program": program,
                                "faculty": faculty
                            }
                            url_element = item_page_soup.select_one('.thumbnail a')
                            url_key =''
                            if url_element:
                                url_key = url_element.get('href')
                            else:

                              continue


                            save_uri_in_json(url_key)
                            save_metadata_in_json(url_key, data_dict)



      except Exception as e:
        print('ERROR: '+url + str(e))




In [6]:
save_uris_pdfs_and_metadata_from_repositorio_unal()

OFFSET: 0


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/url

OFFSET: 20


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/url

OFFSET: 40


...

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/url

OFFSET: 1880


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/url

OFFSET: 1900


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'repositorio.unal.edu.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/url

Se deberían tener 2 archivos:
- Dataset.json
 - Lista de la información preliminar de la tesis (advisor, author, date, description, rtitle, program, faculty), con la uri como llave
- uris.json
 - Lista de las uris (que son las keys de dataset) y un estado pending

In [ ]:
def from_url_to_txt(pdf_url):
  try:
    response = requests.get(pdf_url, verify=False)
    response.raise_for_status()
    pdf_stream = pdfplumber.open(BytesIO(response.content))
    pdf_text = ""
    for page in pdf_stream.pages:
      page_text = page.extract_text()
      pdf_text += page_text
    pdf_stream.close()
  except:
    print('ERROR: '+pdf_url)
  return pdf_text

In [ ]:
def save_content_in_json(uri, raw_content):
  file_name = 'raw_dataset.json'
  if os.path.exists(file_name):
      with open(file_name, 'r', encoding='utf-8') as json_file:
          data = json.load(json_file)
  else:
      data = {"raw": []}


  data["raw"].append({"uri": uri, "raw_content": raw_content})

  with open(file_name, 'w', encoding='utf-8') as json_file:
      json.dump(data, json_file, indent=4, ensure_ascii=False)

In [ ]:
def update_uri_state(uri, new_state):
    file_name = 'uris.json'
    if os.path.exists(file_name):
        with open(file_name, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
        if uri in data["uris"]:
            data["uris"][uri] = new_state
            with open(file_name, 'w', encoding='utf-8') as json_file:
                json.dump(data, json_file, indent=4, ensure_ascii=False)

In [ ]:
def get_pending_uris():
    file_name = 'uris.json'
    pending_uris = []

    if not os.path.exists(file_name):
      uri_list = save_uris_pdfs_and_metadata_from_repositorio_unal()
      return uri_list

    with open(file_name, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        uris_dict = data.get("uris", {})

    for uri, state in uris_dict.items():
        if state == "pending":
            pending_uris.append(uri)

    return pending_uris

def get_error_uris():
    file_name = 'uris.json'
    error_uris = []

    with open(file_name, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        uris_dict = data.get("uris", {})

    for uri, state in uris_dict.items():
        if state == "error":
            error_uris.append(uri)

    return error_uris

CARGAR ARCHIVOS uris.json y raw_dataset.json SI EXISTEN


In [ ]:
uri_list = get_pending_uris()

In [ ]:
len(get_error_uris())

21

In [ ]:
len(uri_list)

0

Se extrae el contenido de cada PDF y se cambia el estado de 'pending' a 'complete'

In [ ]:
def read_pdfs_by_uri_and_update_state():
  for uri in uri_list:
    print('INICIA uri: ' + uri)
    url = 'https://repositorio.unal.edu.co'+uri
    try:
      raw_content = from_url_to_txt(url)
    except:
      update_uri_state(uri, 'error')

    if raw_content != '':
      print('FINALIZA uri: ' + uri+ ' len: '+ str(len(raw_content)))
      save_content_in_json(uri, raw_content)
      update_uri_state(uri, 'complete')
    else:
      print('ERROR URI: '+ uri)
      update_uri_state(uri, 'error')


In [ ]:
read_pdfs_by_uri_and_update_state()

In [ ]:
def get_list_raw_dataset():
  file_name = 'raw_dataset.json'
  raw_dataset = []
  with open(file_name, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

  return data

In [ ]:
#WIP
def extract_content(text, index):
    content_started = False
    content = []

    lines = text.split("\n")
    i = 0
    j =0
    for line in lines:
      if content_started:
            j+=1
            if j==100:
              print('ERROR NO END')
              return ['ERROR']

            if "referencias" in line.strip().lower() or "bibliograf" in line.strip().lower():
              break
            if re.match(r'^\d+\.\d+', line.lstrip()):
              print('salto: '+line.lstrip())
              continue
            line= line.strip().lower().replace('.', '').lstrip('0123456789').rstrip('0123456789')
            line = re.sub(r'\b(?:i|ii|iii|iv|v|vi|vii|viii|ix|x)+\b', '', line, flags=re.IGNORECASE).lstrip().rstrip()

            if line != '' and 'anexo' not in line:
              content.append(line)
      i+=1
      if i == 1000 and not content_started:
        print('ERROR NOT FOUND')
        return ['ERROR']

      if  line.strip().lower().lstrip().rstrip().lstrip('0123456789').endswith("contenido") or line.strip().lower().lstrip().rstrip().lstrip('0123456789').endswith('índice'):
          content_started = True


    content.append(str(index)+ '-------------------------------------------------------')
    return content

In [ ]:
raw_data = get_list_raw_dataset()['raw']


In [ ]:
print(extract_content(raw_data[0]['raw_content']))

['contenido', 'pág', 'resumen', 'lista de figuras', 'lista de tablas', 'introducción', 'estadio alfonso lópez', 'historia', 'ubicación', 'estructura', 'criterios de diseño', 'normas aplicables', 'cancha de futbol', 'iluminación de emergencia', 'ubicación de luminarias', 'diseño de iluminación', 'características de las luminarias', 'factor de mantenimiento', 'resultados de diseño', 'disposición de luminarias', 'iluminación arquitectónica', 'control de iluminación', 'controlador mediante software', 'equipo transmisor', 'equipo receptor', 'conexión luminarias', 'análisis financiero', 'conclusiones y recomendaciones']


In [ ]:
content = []
no_content_uris = []
i=0
c = ['ERROR']
for raw in raw_data:
  i+=1
  if i == 100:
    break
  c = extract_content(raw['raw_content'], i)
  if c[0] == 'ERROR':
    no_content_uris.append(raw['uri'])
  else:
    for item in c:
      if item not in content:
        content.append(item)

In [ ]:
content

In [ ]:
no_content_uris

['/bitstream/handle/unal/84368/1116912174.2023.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/84121/Proyecto%20de%20grado%20EDDP%20Sacha%20Inchi.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/83939/1136883450.2023.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/83548/1037601498.2023.pdf?sequence=4&isAllowed=y',
 '/bitstream/handle/unal/83545/1107076883.2022.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/83297/1023890031.2022.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/83208/1073604762.2023.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/83178/1032419191.2022.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/83159/1052382665.2023.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/83105/80829001-2023.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/83083/1136884823.2022.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/83002/87069558.2022%20.pdf?sequence=2&isAllowed=y',
 '/bitstream/handle/unal/82971/1018463545.2022.pdf?sequence=